In [1]:
"""""""""
import pandas as pd
df = pd.read_csv("new_sample_data.csv", usecols = ["Unnamed: 0","user_id", "restaurant_id", "Rating_x", "title", "clicked"])
df.columns
user_list = []
user_list = list(df["user_id"].unique())
res_list = []
res_list = list(df[df["user_id"] == 0][["title", "clicked"]]["title"].unique())
length = len(res_list)
users_list = []
rest_list = []
rest_name = []
for j in range(len(user_list)):
    res_list = []
    res_list = list(df[df["user_id"] == user_list[j]][["title", "clicked"]]["title"].unique())
    for i in range(len(res_list)):
        resid = list(df[df["title"] == res_list[i]]["restaurant_id"].unique())
        rest_list.append(resid)
        rest_name.append(res_list[i])
        users_list.append(user_list[j])
for i in range(len(rest_list)):
    s = str(rest_list[i][0])
    num = int(s)
    rest_list[i] = num
clicked = []
clicked = [1]*len(rest_list)
import csv
with open("implicit_restaurants.csv", "w", newline = "") as f:
    writer = csv.writer(f)
    l = [["user_id", "restaurant_id", "title", "event_clicked"]]
    writer.writerows(l)
    ll = []
    for i in range(len(rest_list)):
        ll.append([users_list[i], rest_list[i], rest_name[i], clicked[i]])
    writer.writerows(ll)
"""""""""

'\nimport pandas as pd\ndf = pd.read_csv("new_sample_data.csv", usecols = ["Unnamed: 0","user_id", "restaurant_id", "Rating_x", "title", "clicked"])\ndf.columns\nuser_list = []\nuser_list = list(df["user_id"].unique())\nres_list = []\nres_list = list(df[df["user_id"] == 0][["title", "clicked"]]["title"].unique())\nlength = len(res_list)\nusers_list = []\nrest_list = []\nrest_name = []\nfor j in range(len(user_list)):\n    res_list = []\n    res_list = list(df[df["user_id"] == user_list[j]][["title", "clicked"]]["title"].unique())\n    for i in range(len(res_list)):\n        resid = list(df[df["title"] == res_list[i]]["restaurant_id"].unique())\n        rest_list.append(resid)\n        rest_name.append(res_list[i])\n        users_list.append(user_list[j])\nfor i in range(len(rest_list)):\n    s = str(rest_list[i][0])\n    num = int(s)\n    rest_list[i] = num\nclicked = []\nclicked = [1]*len(rest_list)\nimport csv\nwith open("implicit_restaurants.csv", "w", newline = "") as f:\n    write

In [4]:
import scipy.sparse as sparse
import numpy as np
import random
import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import pandas as pd

ModuleNotFoundError: No module named 'implicit'

In [7]:
!pip install implicit

  Using cached implicit-0.4.4.tar.gz (1.1 MB)
  Running setup.py clean for implicit
Failed to build implicit
    Running setup.py install for implicit: started
    Running setup.py install for implicit: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'c:\users\lenovo\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Lenovo\\AppData\\Local\\Temp\\pip-install-lk8qgle8\\implicit\\setup.py'"'"'; __file__='"'"'C:\\Users\\Lenovo\\AppData\\Local\\Temp\\pip-install-lk8qgle8\\implicit\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Lenovo\AppData\Local\Temp\pip-wheel-y7b30fey'
       cwd: C:\Users\Lenovo\AppData\Local\Temp\pip-install-lk8qgle8\implicit\
  Complete output (26 lines):
  Failed to find CUDA toolkit. Building without GPU acceleration.
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\implicit
  copying implicit\als.py -> build\lib.win-amd64-3.8\implicit
  copying implicit\approximate

In [23]:
df_1 = pd.read_csv("implicit_restaurants.csv")
df_1 = df_1.drop_duplicates()

In [24]:
grouped_df = df_1.groupby(['user_id', 'restaurant_id', 'title']).sum().reset_index()

In [25]:
grouped_df

,user_id,restaurant_id,title,event_clicked
0,0,217,A&W Restaurant,1
1,0,363,Chartreuse Moose Cappuccino Bar & Bistro,1
2,0,370,Jake's,1
3,0,373,P Bass Fish and Chips,1
4,1,204,Canadian 2 For 1 Pizza,1
...,...,...,...,...
87,19,204,Canadian 2 For 1 Pizza,1
88,19,217,A&W Restaurant,1
89,19,370,Jake's,1
90,19,373,P Bass Fish and Chips,1


In [26]:
grouped_df['title'] = grouped_df['title'].astype("category")
grouped_df['user_id'] = grouped_df['user_id'].astype("category")
grouped_df['restaurant_id'] = grouped_df['restaurant_id'].astype("category")
grouped_df['user_id'] = grouped_df['user_id'].cat.codes
grouped_df['restaurant_id'] = grouped_df['restaurant_id'].cat.codes

sparse_content_person = sparse.csr_matrix((grouped_df['event_clicked'].astype(float), (grouped_df['restaurant_id'], grouped_df['user_id'])))
sparse_person_content = sparse.csr_matrix((grouped_df['event_clicked'].astype(float), (grouped_df['user_id'], grouped_df['restaurant_id'])))

model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

alpha = 15
data = (sparse_content_person * alpha).astype('double')
model.fit(data)

In [27]:
content_id = 3
n_similar = 10

person_vecs = model.user_factors
content_vecs = model.item_factors

content_norms = np.sqrt((content_vecs * content_vecs).sum(axis=1))
scores = content_vecs.dot(content_vecs[content_id]) / content_norms
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
similar = sorted(zip(top_idx, scores[top_idx] / content_norms[content_id]), key=lambda x: -x[1])
for content in similar:
    idx, score = content
    print(grouped_df.title.loc[grouped_df.restaurant_id == idx].iloc[0])

A&W Restaurant
El Caballo Restaurant
Jake's
Subway
Chartreuse Moose Cappuccino Bar & Bistro
P Bass Fish and Chips
Smitty's 100 Mile House
Tim Hortons
Farrier pub
The Great Wok


In [28]:
grouped_df[grouped_df["restaurant_id"] == 1]["title"]

14    Tim Hortons
54    Tim Hortons
82    Tim Hortons
Name: title, dtype: category
Categories (15, object): ['A&W Restaurant', 'BJ's Donuts & Eatery', 'Blue Sky Chinese Restaurant', 'Canadian 2 For 1 Pizza', ..., 'Subway', 'Sushi and Noodle', 'The Great Wok', 'Tim Hortons']

In [29]:
def recommend(person_id, sparse_person_content, person_vecs, content_vecs, num_contents=10):
    # Get the interactions scores from the sparse person content matrix
    person_interactions = sparse_person_content[person_id,:].toarray()
    # Add 1 to everything, so that articles with no interaction yet become equal to 1
    person_interactions = person_interactions.reshape(-1) + 1
    # Make articles already interacted zero
    person_interactions[person_interactions > 1] = 0
    # Get dot product of person vector and all content vectors
    rec_vector = person_vecs[person_id,:].dot(content_vecs.T).toarray()
    
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    # Content already interacted have their recommendation multiplied by zero
    recommend_vector = person_interactions * rec_vector_scaled
    # Sort the indices of the content into order of best recommendations
    content_idx = np.argsort(recommend_vector)[::-1][:num_contents]
    
    # Start empty list to store titles and scores
    titles = []
    scores = []

    for idx in content_idx:
        # Append titles and scores to the list
        titles.append(grouped_df.title.loc[grouped_df.restaurant_id == idx].iloc[0])
        scores.append(recommend_vector[idx])

    recommendations = pd.DataFrame({'title': titles, 'score': scores})

    return recommendations
    
# Get the trained person and content vectors. We convert them to csr matrices
person_vecs = sparse.csr_matrix(model.user_factors)
content_vecs = sparse.csr_matrix(model.item_factors)

In [37]:
person_id = 0
recommendations = recommend(person_id, sparse_person_content, person_vecs, content_vecs)
print("Because you visited", grouped_df[grouped_df["user_id"] == person_id]["title"][0])
print(recommendations)

Because you visited A&W Restaurant
                         title     score
0                       Subway  0.024795
1                  Tim Hortons  0.022401
2                The Great Wok  0.018646
3       Canadian 2 For 1 Pizza  0.017721
4                  Farrier pub  0.016547
5         BJ's Donuts & Eatery  0.016525
6             Sushi and Noodle  0.014690
7  Blue Sky Chinese Restaurant  0.007333
8      Smitty's 100 Mile House  0.005884
9        El Caballo Restaurant  0.003575


In [38]:
import random

def make_train(ratings, pct_test = 0.2):
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of item,user index into list

    
    random.seed(0) # Set the random seed to zero for reproducibility
    
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of item-user pairs without replacement

    content_inds = [index[0] for index in samples] # Get the item row indices

    person_inds = [index[1] for index in samples] # Get the user column indices

    
    training_set[content_inds, person_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    
    return training_set, test_set, list(set(person_inds))

In [39]:
content_train, content_test, content_persons_altered = make_train(sparse_content_person, pct_test = 0.2)

In [40]:
def auc_score(predictions, test):
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [41]:
def calc_mean_auc(training_set, altered_persons, predictions, test_set):
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_contents = np.array(test_set.sum(axis = 1)).reshape(-1) # Get sum of item iteractions to find most popular
    content_vecs = predictions[1]
    for person in altered_persons: # Iterate through each user that had an item altered
        training_column = training_set[:,person].toarray().reshape(-1) # Get the training set column
        zero_inds = np.where(training_column == 0) # Find where the interaction had not yet occurred
        
        # Get the predicted values based on our user/item vectors
        person_vec = predictions[0][person,:]
        pred = person_vec.dot(content_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[:,person].toarray()[zero_inds,0].reshape(-1)
        
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_contents[zero_inds] # Get the item popularity for our chosen items
        
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))

In [42]:
calc_mean_auc(content_train, content_persons_altered,
              [person_vecs, content_vecs.T], content_test) 

(1.0, 0.734)